In [ ]:
!pip install tensorflow
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from google.colab import files
from nltk.tokenize import word_tokenize

from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

from tensorflow.keras.layers import Dropout

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
uploaded = files.upload()

Saving amazon_cells_labelled.txt to amazon_cells_labelled.txt
Saving imdb_labelled.txt to imdb_labelled.txt
Saving yelp_labelled.txt to yelp_labelled.txt


#Cargue de Datos

In [ ]:
# Cargue de archivos
amazon_data = pd.read_csv('amazon_cells_labelled.txt', delimiter='\t', header=None, names=['text', 'label'])
imdb_data = pd.read_csv('imdb_labelled.txt', delimiter='\t', header=None, names=['text', 'label'])
yelp_data = pd.read_csv('yelp_labelled.txt', delimiter='\t', header=None, names=['text', 'label'])

# Combinar datasets
data = pd.concat([amazon_data, imdb_data, yelp_data], ignore_index=True)

#NLTK

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

#Tokenizacion y limpieza de datos

In [ ]:
def clean_text(text):
    # Tokenización
    tokens = word_tokenize(text)

    # Convertir a minúsculas y eliminar stopwords
    cleaned_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]

    # Unir los tokens limpios en una cadena
    return ' '.join(cleaned_tokens)

In [ ]:
data['cleaned_text'] = data['text'].apply(clean_text)

data.head()

#Preparación de los datos

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = data['cleaned_text']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=True, random_state=154)

In [ ]:
# Vectorización de los textos
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

#DummyClassifier

In [ ]:
# Implementar DummyClassifier
dummy_clf = DummyClassifier(strategy='most_frequent')

# Entrenar el modelo
dummy_clf.fit(X_train_vectorized, y_train)

In [ ]:
# Hacer predicciones
y_pred = dummy_clf.predict(X_test_vectorized)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

#RNN

In [ ]:
# Tokenización y secuenciación
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding para asegurar que todas las secuencias tengan el mismo tamaño
maxlen = 100
X_train_padded = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_seq, maxlen=maxlen)

In [ ]:
def create_model():
  model = Sequential()
  model.add(Embedding(10000, 16, name='embedding'))
  model.add(SimpleRNN(8))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
rnnModel = create_model()

In [ ]:
rnnModel.fit(X_train_padded, y_train, epochs=100, verbose=1)